In [43]:
import pandas as pd
import io
import os
from google.colab import drive
import numpy as np
from datetime import datetime

In [44]:
# This will allow us to save files in Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#### 1) First, we will define a function that will allow us to extract weather data from Canada Gov

In [45]:
def getHourlyData(stationID, year, month):
    base_url = "http://climate.weather.gc.ca/climate_data/bulk_data_e.html?"
    query_url = "format=csv&stationID={}&Year={}&Month={}&timeframe=1".format(stationID, year, month)
    api_endpoint = base_url + query_url
    return pd.read_csv(api_endpoint, skiprows=0)

#### 2) We get the weather for only one weather station McTavish (ID=30165)

In [46]:
stationID = 5415
year_list = [2005,2006,2007,2008]

frames_1 = []
for year in year_list:
  for month in range(12):
    df = getHourlyData(stationID, year, month+1)
    frames_1.append(df)

stationID = 30165
year_list = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2019,2020]

frames_2 = []
for year in year_list:
  for month in range(12):
    df = getHourlyData(stationID, year, month+1)
    frames_2.append(df)

In [47]:
weather_data_1 = pd.concat(frames_1)
weather_data_2 = pd.concat(frames_2)
frames = [weather_data_1, weather_data_2]
weather_data = pd.concat(frames)

#### 3) Recode time into shifts

In [48]:
# Data cleaning
climate_raw = weather_data.copy()
climate_raw['Shift'] = climate_raw['Time'].apply(lambda x: 1 if x<'08:00' else 2 if  x<'16:00' else 3)
climate_raw['Date']  = climate_raw['Date/Time'].str.slice(start=0, stop=10)

# Keep useful columns
climate = climate_raw[[ 'Date', 'Shift', 'Temp (°C)', 'Rel Hum (%)', 'Wind Dir (10s deg)', 'Wind Spd (km/h)', 'Stn Press (kPa)']]

# Rename columns
climate = climate.rename(columns={'Date': 'Date', 'Shift': 'Shift', 'Temp (°C)': 'Temp_DC', 'Rel Hum (%)': 'Humid_percent', 'Wind Dir (10s deg)': 'Win_Dir', 
                                  'Wind Spd (km/h)': 'Wind_Speed', 'Stn Press (kPa)': 'Stn_Press'})

# Use aggregate functions
climate_mean = climate.groupby(['Date','Shift']).mean()
climate_min  = climate.groupby(['Date','Shift']).min()
climate_max  = climate.groupby(['Date','Shift']).max()

climate_mean = climate_mean.rename(columns={'Temp_DC': 'Temp_DC_Mean', 'Humid_percent': 'Humid_percent_Mean', 'Win_Dir': 'Win_Dir_Mean',
                                            'Wind_Speed': 'Wind_Speed_Mean', 'Stn_Press': 'Stn_Press_Mean'})
climate_max = climate_max.rename(columns={'Temp_DC': 'Temp_DC_Max', 'Humid_percent': 'Humid_percent_Max', 'Win_Dir': 'Win_Dir_Max', 
                                          'Wind_Speed': 'Wind_Speed_Max', 'Stn_Press': 'Stn_Press_Max'})
climate_min = climate_min.rename(columns={'Temp_DC': 'Temp_DC_Min', 'Humid_percent': 'Humid_percent_Min', 'Win_Dir': 'Win_Dir_Min', 
                                          'Wind_Speed': 'Wind_Speed_Min', 'Stn_Press': 'Stn_Press_Min'})

# Join data
climate_final =climate_mean.join(climate_max)
climate_final =climate_final.join(climate_min)

In [49]:
# Check data
climate_final = climate_final.loc['2005-01-01':'2020-06-30']
climate_final.head()

Temp_DC_Mean  ...  Stn_Press_Min
Date       Shift                ...               
2005-01-01 1            5.2250  ...         100.47
           2            0.6250  ...         101.60
           3           -5.5750  ...         102.86
2005-01-02 1          -13.8750  ...         103.69
           2          -13.9375  ...         102.19

[5 rows x 15 columns]

#### 4) Output data to One Drive

In [50]:
climate_final.to_csv('/content/drive/My Drive/Data/YCBS-299/Weather_data_2005_2020.csv')